In [1]:
import sys
sys.path.append("../../")

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from model.metrics import *
from data.utils import load_img

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=110)

# Choose GPU Device

# Set Experiment Name

In [4]:
data_pth = Path('/home/alaa/Dropbox/BPHO Staff/USF')  # path to dataset
dir1 = f'EM/testing/LR-PSSR/real-world_SEM/'  # input images
dir2 = f'EM/testing/LR-MAE/real-world_SEM/'  # target images
dir3 = f'EM/testing/LR-Bilinear/real-world_SEM/'  # target images
targ_dir = f'EM/testing/HR/real-world_SEM/'

In [5]:
# Modify accordingly
pth1 = data_pth/dir1
pth2 = data_pth/dir2
pth3 = data_pth/dir3
targ_pth = data_pth/targ_dir

dir1_files = sorted(list(pth1.glob('*.tif')))

dir2_files = sorted(list(pth2.glob('*.tif')))

dir3_files = sorted(list(pth3.glob('*.tif')))


targ_files = sorted(list(targ_pth.glob('*.tif')))

print('Processing '+str(len(dir1_files))+' files...')

Processing 42 files...


In [6]:
# Set size for visualizations
fig_size = plt.rcParams["figure.figsize"]  # Get current size
print("Current size:", fig_size)
 
# Set figure width to 12 and height to 9
fig_size[0] = 30
fig_size[1] = 24
plt.rcParams["figure.figsize"] = fig_size

Current size: [6.0, 4.0]


In [7]:
def visualize_sample(img_fname1, img_fname2, targ_fname, title1="PSSR", title2="Target", title3="MAE"):
    pssr_im = load_img(img_fname1) # load image to tensor
    targ_im = load_img(targ_fname)
    mae_im = load_img(img_fname2)
    pssr_psnr = np.abs(psnr(torch.tensor(pssr_im), torch.tensor(targ_im)))
    pssr_ssim = np.abs(ssim(torch.tensor(targ_im).unsqueeze(0), torch.tensor(pssr_im).unsqueeze(0)))
    mae_psnr = np.abs(psnr(torch.tensor(mae_im), torch.tensor(targ_im)))
    mae_ssim = np.abs(ssim(torch.tensor(targ_im).unsqueeze(0), torch.tensor(mae_im).unsqueeze(0)))
    f, axarr = plt.subplots(1,3) # create visualizations
    axarr[0].imshow(np.squeeze(pssr_im), cmap=plt.cm.gray) # visualize image tensor
    axarr[0].set_title(title1)
    axarr[0].set_xlabel(f"PSNR: {pssr_psnr:.2f}, SSIM: {pssr_ssim:.2f}")
    axarr[1].imshow(np.squeeze(targ_im), cmap=plt.cm.gray) # visualize original image file
    axarr[1].set_title(title2)
    axarr[2].imshow(np.squeeze(pssr_im), cmap=plt.cm.gray) # visualize image tensor
    axarr[2].set_title(title3)
    axarr[2].set_xlabel(f"PSNR: {mae_psnr:.2f}, SSIM: {mae_ssim:.2f}")
    plt.show() # show visualization

In [8]:
@interact
def show_sample(sample=10):
    return visualize_sample(dir1_files[sample], dir2_files[sample], targ_files[sample])

interactive(children=(IntSlider(value=10, description='sample', max=30, min=-10), Output()), _dom_classes=('wi…

In [9]:
def evaluate_model(pred_files, hr_files):
    psnr_scores = {}
    ssim_scores = {}
    for pred_fname, targ_fname in zip(pred_files, hr_files):
        pred_img = load_img(pred_fname)
        targ_img = load_img(targ_fname)
        
        psnr_score = np.abs(psnr(torch.tensor(pred_img), torch.tensor(targ_img)))
        psnr_scores[targ_fname] = psnr_score
        
        ssim_score = ssim(torch.tensor(targ_img).unsqueeze(0), torch.tensor(pred_img).unsqueeze(0))
        ssim_scores[targ_fname] = ssim_score
    return psnr_scores, ssim_scores

In [10]:
mae_psnr, mae_ssim = evaluate_model(dir2_files, targ_files)

In [11]:
mse_psnr, mse_ssim = evaluate_model(dir1_files, targ_files)

In [12]:
bilin_psnr, bilin_ssim = evaluate_model(dir3_files, targ_files)

In [13]:
# MSE PSSR PSNR MEAN
np.mean([mse_psnr[f] for f in mse_psnr.keys()])

23.384266

In [14]:
# MSE PSSR SSIM MEAN
np.mean([mse_ssim[f] for f in mse_ssim.keys()])

0.3262943

In [15]:
# MAE PSSR PSNR MEAN
np.mean([mae_psnr[f] for f in mae_psnr.keys()])

21.169094

In [16]:
# MAE PSSR SSIM MEAN
np.mean([mae_ssim[f] for f in mae_ssim.keys()])

0.23968537

In [17]:
# Bilinear PSNR MEAN
np.mean([bilin_psnr[f] for f in bilin_psnr.keys()])

21.163029

In [18]:
# Bilinear SSIM MEAN
np.mean([bilin_ssim[f] for f in bilin_ssim.keys()])

0.25493416